In [1]:
import gymnasium as gym
import longshot
from longshot.circuit import FormulaType, Literals
from longshot.gym import FlattenSequence, LambdaMixedReward, XORAction, SearchForXOR

from IPython import display
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from torch.utils.tensorboard import SummaryWriter
import numpy as np

from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import TensorBoardOutputFormat
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.monitor import Monitor

2025-05-03 12:38:45.433927: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-03 12:38:45.439901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746275925.446795  338614 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746275925.448894  338614 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746275925.454496  338614 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
%load_ext tensorboard
%matplotlib inline

In [3]:
num_vars = 5
r_alpha = 1.0
r_beta = 1.0
r_gamma = 0.
r_eps = 1e-5

In [4]:
env = gym.make("longshot/avgQ-d2-formula", n=num_vars, ftype=FormulaType.Disjunctive, mono=False)
env = SearchForXOR(env)
env = LambdaMixedReward(env, alpha=r_alpha, beta=r_beta, gamma=r_gamma, eps=r_eps)
env = Monitor(env)

In [5]:
policy_kwargs = {
    "net_arch": [256, 256, 128]
}
model = PPO("MlpPolicy", env, tensorboard_log='runs', device="cpu", policy_kwargs=policy_kwargs)
model.learn(total_timesteps=int(1e6), progress_bar=True)
model.save(f"models/sb3_ppo_longshot_n{num_vars}_s4x")

Output()

KeyboardInterrupt: 

In [ ]:
env = gym.make("longshot/avgQ-d2-formula", n=num_vars, ftype=FormulaType.Disjunctive, mono=False)
env = SearchForXOR(env)
env = LambdaMixedReward(env, alpha=r_alpha, beta=r_beta, gamma=r_gamma, eps=r_eps)
env = Monitor(env)
model = PPO.load(f"models/sb3_ppo_longshot_n{num_vars}_s4x", device="cpu")

for _ in range(10):
    obs, info = env.reset()
    tr = 0.
    while True:
        action, _states = model.predict(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        tr += reward
        
        if terminated or truncated:
            print(obs, tr, info)
            break
            
    print(obs, tr, info)